Train ViT model

In [4]:
# import os
# import av
# import torch
# import numpy as np
# import random
# import json
# import time
# import subprocess
# import psutil
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from transformers import ViTForImageClassification, ViTImageProcessor
# from collections import defaultdict
# from torch.utils.data import DataLoader, Dataset

# # 定义配置
# config = {
#     "model_name": "google/vit-base-patch16-224-in21k",
#     "image_processor_name": "google/vit-base-patch16-224-in21k",
#     "num_samples": 10000,
#     "num_classes": 400,  # For flexible dataset input
#     "num_samples_per_class": 25,  # For flexible dataset input
#     "video_list_path": "archive/kinetics400_val_list_videos.txt",
#     "video_directory": "archive/zoom_blur",
#     "use_exact": True
# }

# # 加载预训练的ViT模型和处理器
# model = ViTForImageClassification.from_pretrained(config["model_name"], num_labels=400)
# image_processor = ViTImageProcessor.from_pretrained(config["image_processor_name"])

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # 读取视频文件和标签
# video_labels = defaultdict(list)
# with open(config["video_list_path"], "r") as f:
#     for line in f:
#         name, label = line.strip().split()
#         video_labels[int(label)].append(name)

# # 准备样本
# sampled_files = []
# true_labels = []
# selected_classes = random.sample(list(video_labels.keys()), config["num_classes"])
# for cls in selected_classes:
#     sampled_files.extend(random.sample(video_labels[cls], config["num_samples_per_class"]))
#     true_labels.extend([cls] * config["num_samples_per_class"])

# # 自定义数据集类
# class Kinetics400Dataset(Dataset):
#     def __init__(self, video_files, labels, image_processor):
#         self.video_files = video_files
#         self.labels = labels
#         self.image_processor = image_processor

#     def __len__(self):
#         return len(self.video_files)

#     def __getitem__(self, idx):
#         video_file = self.video_files[idx]
#         label = self.labels[idx]
#         container = av.open(os.path.join(config["video_directory"], video_file))
#         frame = container.decode(video=0).__next__().to_image()  # 获取第一帧
#         inputs = self.image_processor(images=frame, return_tensors="pt")
#         inputs = {k: v.squeeze(0) for k, v in inputs.items()}
#         return inputs['pixel_values'], torch.tensor(label)

# # 准备数据集和数据加载器
# dataset = Kinetics400Dataset(sampled_files, true_labels, image_processor)
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# # 微调模型
# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
# criterion = torch.nn.CrossEntropyLoss()

# model.train()
# for epoch in range(5):  # 你可以根据需要调整epoch数量
#     for batch in dataloader:
#         images, labels = batch
#         images = images.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images).logits
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# # 预测函数
# model.eval()
# def predict_video(video_file):
#     container = av.open(os.path.join(config["video_directory"], video_file))
#     frame = container.decode(video=0).__next__().to_image()  # 获取第一帧
#     inputs = image_processor(images=frame, return_tensors="pt")
#     inputs = {k: v.to(device) for k, v in inputs.items()}

#     with torch.no_grad():
#         outputs = model(**inputs).logits
#         predicted_label = torch.argmax(outputs, dim=-1).item()
#     return predicted_label

# # 对样本进行预测并评估
# predictions = [predict_video(video_file) for video_file in sampled_files]

# accuracy = accuracy_score(true_labels, predictions)
# precision = precision_score(true_labels, predictions, average='weighted')
# recall = recall_score(true_labels, predictions, average='weighted')
# f1 = f1_score(true_labels, predictions, average='weighted')

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")


/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 5.392894744873047
Epoch 2, Loss: 4.374405860900879
Epoch 3, Loss: 2.4989871978759766
Epoch 4, Loss: 2.265422821044922
Epoch 5, Loss: 0.9371886253356934
Accuracy: 0.8917
Precision: 0.9100
Recall: 0.8917
F1 Score: 0.8870


Train ViT model and save the finetune results

In [7]:
import os
import av
import torch
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import ViTForImageClassification, ViTImageProcessor
from collections import defaultdict
from torch.utils.data import DataLoader, Dataset

# 定义配置
config = {
    "model_name": "google/vit-base-patch16-224-in21k",
    "image_processor_name": "google/vit-base-patch16-224-in21k",
    "num_samples": 10000,
    "num_classes": 400,  # For flexible dataset input
    "num_samples_per_class": 25,  # For flexible dataset input
    "video_list_path": "archive/kinetics400_val_list_videos.txt",
    "video_directory": "archive/zoom_blur",
    "use_exact": True
}

# 加载预训练的ViT模型和处理器
model = ViTForImageClassification.from_pretrained(config["model_name"], num_labels=400)
image_processor = ViTImageProcessor.from_pretrained(config["image_processor_name"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 读取视频文件和标签
video_labels = defaultdict(list)
with open(config["video_list_path"], "r") as f:
    for line in f:
        name, label = line.strip().split()
        video_labels[int(label)].append(name)

# 准备样本
sampled_files = []
true_labels = []
selected_classes = random.sample(list(video_labels.keys()), config["num_classes"])
for cls in selected_classes:
    sampled_files.extend(random.sample(video_labels[cls], config["num_samples_per_class"]))
    true_labels.extend([cls] * config["num_samples_per_class"])

# 自定义数据集类
class Kinetics400Dataset(Dataset):
    def __init__(self, video_files, labels, image_processor):
        self.video_files = video_files
        self.labels = labels
        self.image_processor = image_processor

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_file = self.video_files[idx]
        label = self.labels[idx]
        container = av.open(os.path.join(config["video_directory"], video_file))
        frame = container.decode(video=0).__next__().to_image()  # 获取第一帧
        inputs = self.image_processor(images=frame, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs['pixel_values'], torch.tensor(label)

# 准备数据集和数据加载器
dataset = Kinetics400Dataset(sampled_files, true_labels, image_processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 微调模型
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(5):  # 你可以根据需要调整epoch数量
    for batch in dataloader:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# 保存微调后的模型
model_save_path = "finetuned_vit_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# 定义推理函数
def predict_video(video_file, model, image_processor):
    container = av.open(video_file)
    frame = container.decode(video=0).__next__().to_image()  # 获取第一帧
    inputs = image_processor(images=frame, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs).logits
        predicted_label = torch.argmax(outputs, dim=-1).item()
        probabilities = torch.nn.functional.softmax(outputs, dim=-1)
        predicted_score = probabilities[0, predicted_label].item()

    return predicted_label, predicted_score



Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 5.6305084228515625
Epoch 2, Loss: 4.717484474182129
Epoch 3, Loss: 3.345651388168335
Epoch 4, Loss: 2.8434462547302246
Epoch 5, Loss: 0.5280552506446838
Model saved to finetuned_vit_model.pth


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 0.5400
Precision: 0.5517
Recall: 0.5400
F1 Score: 0.5300
Video file: archive/videos_val/B58eZb6k1OI.mp4
True label: 144
Predicted label: 184
Predicted score: 0.0964
---------
Video file: archive/videos_val/mCaCagGJK2I.mp4
True label: 47
Predicted label: 123
Predicted score: 0.0344
---------
Video file: archive/videos_val/7YUSAxdW-e8.mp4
True label: 7
Predicted label: 297
Predicted score: 0.0310
---------
Video file: archive/videos_val/ctxQe5W2Bf0.mp4
True label: 166
Predicted label: 166
Predicted score: 0.0834
---------
Video file: archive/videos_val/1pEaxLFdOZY.mp4
True label: 179
Predicted label: 207
Predicted score: 0.0707
---------
Video file: archive/videos_val/iDj25an_tRY.mp4
True label: 340
Predicted label: 62
Predicted score: 0.0598
---------
Video file: archive/videos_val/PmgfU9ocx5A.mp4
True label: 193
Predicted label: 389
Predicted score: 0.0731
---------
Video file: archive/videos_val/qqrnC0WAZlI.mp4
True label: 101
Predicted label: 140
Predicted score: 0.1192
---

/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# 重新加载模型进行推理

In [9]:
# 重新加载模型进行推理
model = ViTForImageClassification.from_pretrained(config["model_name"], num_labels=400)
model.load_state_dict(torch.load(model_save_path))
model.to(device)
model.eval()

# 自动读取指定目录下的随机100个视频文件并进行推理
def get_label_for_video(video_file, video_labels):
    for label, files in video_labels.items():
        if os.path.basename(video_file) in files:
            return label
    return None

video_directory = "archive/videos_val"
video_files = [os.path.join(video_directory, f) for f in os.listdir(video_directory) if f.endswith('.mp4')]
random_video_files = random.sample(video_files, min(100, len(video_files)))  # 随机选择100个视频文件

# 进行推理并评估
true_labels = []
predicted_labels = []
predicted_scores = []

for video_file in random_video_files:
    true_label = get_label_for_video(video_file, video_labels)
    if true_label is not None:
        predicted_label, predicted_score = predict_video(video_file, model, image_processor)
        true_labels.append(true_label)
        predicted_labels.append(predicted_label)
        predicted_scores.append(predicted_score)
    else:
        print(f"Label for video file {video_file} not found.")

# 计算性能指标
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# 打印结果
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# 打印详细结果
for i, video_file in enumerate(random_video_files):
    print(f"Video file: {video_file}")
    print(f"True label: {true_labels[i]}")
    print(f"Predicted label: {predicted_labels[i]}")
    print(f"Predicted score: {predicted_scores[i]:.4f}")
    print("---------")


/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 0.5000
Precision: 0.4483
Recall: 0.5000
F1 Score: 0.4650
Video file: archive/videos_val/cs4UM8n4-Jc.mp4
True label: 1
Predicted label: 41
Predicted score: 0.0251
---------
Video file: archive/videos_val/K0DOqOk-b0c.mp4
True label: 82
Predicted label: 154
Predicted score: 0.0171
---------
Video file: archive/videos_val/QLG7JT6SrXo.mp4
True label: 389
Predicted label: 386
Predicted score: 0.1484
---------
Video file: archive/videos_val/iCIFDPQqA1c.mp4
True label: 276
Predicted label: 276
Predicted score: 0.0645
---------
Video file: archive/videos_val/L_KiHjIcRPY.mp4
True label: 241
Predicted label: 241
Predicted score: 0.2000
---------
Video file: archive/videos_val/fxX73D9AV88.mp4
True label: 176
Predicted label: 176
Predicted score: 0.1681
---------
Video file: archive/videos_val/qgHDfJjjkh0.mp4
True label: 339
Predicted label: 340
Predicted score: 0.4870
---------
Video file: archive/videos_val/ryB5YcK2PMY.mp4
True label: 38
Predicted label: 16
Predicted score: 0.0877
-----

/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
